### Loading and visualizing data

In [1]:
from os import listdir

work_dir = listdir('aclImdb/train/pos/')
positive_reviews = []

for review in work_dir:
    with open(f'aclImdb/train/pos/{review}','r') as f:
        positive_reviews.append(f.read())

work_dir = listdir('aclImdb/train/neg/')
negative_reviews = []
for review in work_dir:
    with open(f'aclImdb/train/neg/{review}','r') as f:
        negative_reviews.append(f.read())


In [2]:
negative_labels = [0 for review in negative_reviews] 
positive_labels = [1 for review in positive_reviews] 
reviews = positive_reviews + negative_reviews
labels = positive_labels + negative_labels

In [26]:
from keras.preprocessing.text import Tokenizer

vocabulary_size = 500
t = Tokenizer(num_words= vocabulary_size)

t.fit_on_texts(reviews) 
encoded_reviews = t.texts_to_sequences(reviews)

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
                                                    encoded_reviews, labels,
                                                    test_size=0.3, random_state=1
                                                   )


### Processing reviews

In [28]:
from keras.preprocessing import sequence
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)


In [29]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
embedding_size=32
model=Sequential()
model.add(Embedding(2000, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 32)           64000     
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 117,301
Trainable params: 117,301
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [32]:
model.fit(X_train, y_train, validation_split=0.20,batch_size=500, epochs=10)

Train on 14000 samples, validate on 3500 samples
Epoch 1/10
14000/14000 [==============================] - 54s 4ms/step - loss: 0.4340 - accuracy: 0.8056 - val_loss: 0.4198 - val_accuracy: 0.8120
Epoch 2/10
14000/14000 [==============================] - 56s 4ms/step - loss: 0.3931 - accuracy: 0.8291 - val_loss: 0.3983 - val_accuracy: 0.8200
Epoch 3/10
14000/14000 [==============================] - 56s 4ms/step - loss: 0.3733 - accuracy: 0.8398 - val_loss: 0.3917 - val_accuracy: 0.8223
Epoch 4/10
14000/14000 [==============================] - 57s 4ms/step - loss: 0.3701 - accuracy: 0.8437 - val_loss: 0.3988 - val_accuracy: 0.8203
Epoch 5/10
14000/14000 [==============================] - 57s 4ms/step - loss: 0.3734 - accuracy: 0.8421 - val_loss: 0.3996 - val_accuracy: 0.8251
Epoch 6/10
14000/14000 [==============================] - 58s 4ms/step - loss: 0.3538 - accuracy: 0.8492 - val_loss: 0.3852 - val_accuracy: 0.8351
Epoch 7/10
14000/14000 [==============================] - 58s 4ms/ste

In [33]:
scores = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scores[1])

Test accuracy: 0.8345333337783813


In [35]:
model.predict(X_test)

array([[0.02807844],
       [0.9667443 ],
       [0.03876644],
       ...,
       [0.09576827],
       [0.11786565],
       [0.8553895 ]], dtype=float32)